In [1]:
#Importing
import  requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import matplotlib.pyplot as plt
import os

In [2]:
# Fetch the financial results page
r = requests.get('https://investors.coca-colacompany.com/financial-information/financial-results')
soup = bs(r.text)
columns = soup.find_all(class_='link', title = 'XBRL Spreadsheet')
columns

[<a class="link" href="https://content.equisolve.net/sec/0000021344-24-000019/Financial_Report.xlsx" target="_blank" title="XBRL Spreadsheet"><span class="eqicon-PDF"></span> XLS</a>,
 <a class="link" href="https://content.equisolve.net/sec/0000021344-24-000009/Financial_Report.xlsx" target="_blank" title="XBRL Spreadsheet"><span class="eqicon-PDF"></span> XLS</a>,
 <a class="link" href="https://content.equisolve.net/sec/0000021344-23-000060/Financial_Report.xlsx" target="_blank" title="XBRL Spreadsheet"><span class="eqicon-PDF"></span> XLS</a>,
 <a class="link" href="https://content.equisolve.net/sec/0000021344-23-000048/Financial_Report.xlsx" target="_blank" title="XBRL Spreadsheet"><span class="eqicon-PDF"></span> XLS</a>,
 <a class="link" href="https://s3.amazonaws.com/content.stockpr.com/sec/0000021344-23-000024/Financial_Report.xlsx" target="_blank" title="XBRL Spreadsheet"><span class="eqicon-PDF"></span> XLS</a>,
 <a class="link" href="https://s3.amazonaws.com/content.stockpr.c

In [3]:
links = [column.get('href') for column in columns[2: -9]]
links

['https://content.equisolve.net/sec/0000021344-23-000060/Financial_Report.xlsx',
 'https://content.equisolve.net/sec/0000021344-23-000048/Financial_Report.xlsx',
 'https://s3.amazonaws.com/content.stockpr.com/sec/0000021344-23-000024/Financial_Report.xlsx',
 'https://s3.amazonaws.com/content.stockpr.com/sec/0000021344-23-000011/Financial_Report.xlsx',
 'https://s3.amazonaws.com/content.stockpr.com/sec/0000021344-22-000042/Financial_Report.xlsx',
 'https://s3.amazonaws.com/content.stockpr.com/sec/0000021344-22-000034/Financial_Report.xlsx',
 'https://s3.amazonaws.com/content.stockpr.com/sec/0000021344-22-000016/Financial_Report.xlsx',
 'https://s3.amazonaws.com/content.stockpr.com/sec/0000021344-22-000009/Financial_Report.xlsx',
 'https://s3.amazonaws.com/content.stockpr.com/sec/0000021344-21-000030/Financial_Report.xlsx',
 'https://s3.amazonaws.com/content.stockpr.com/sec/0000021344-21-000024/Financial_Report.xlsx',
 'https://s3.amazonaws.com/content.stockpr.com/sec/0000021344-21-00001

In [4]:
def downloads(download_link):
    try:
        df = pd.read_excel(download_link, engine='openpyxl', sheet_name=2)  # Read sheet 2
        
        return df
    
    except Exception as e:
        print(f"Error downloading/processing link: {download_link}. Error: {e}")
        
        return None  # Handle download errors gracefully

In [5]:
datas = []

for link in links:
    print(f'Downloading link --> {link}')
    df = downloads(link)
    if df is not None:  # Skip dataframes with download/processing errors
        datas.append(df)

In [6]:
def save_dataframe(df, filename):
    try:
        df.to_csv(filename, index=False)  # Save as CSV, without index
        print(f"Successfully saved dataframe to: {filename}")
    except Exception as e:
        print(f"Error saving dataframe: {e}")
        
# Check if any dataframes failed to download/process
if not all(df is not None for df in datas):
    print("Warning: Some dataframes may have download/processing errors.")

# Create a directory to save dataframes
data_dir = "downloaded_dataframes" 
os.makedirs(data_dir, exist_ok=True)  # Create directory if it doesn't exist

years_quarter = []
for y in ('2023', '2022', '2021', '2020', '2019', '2018', '2017', '2016', '2015', '2014', '2013'):
    for q in ('_Q4', '_Q3', '_Q2', '_Q1'):
        year = y + q
         # Stop appending if reaching 2013 Q4
        if year == "2013_Q4":
            years_quarter.append(year)
            break  # Exit the inner loop after appending 2013 Q4
        else:
            years_quarter.append(year)
            
        
# Save individual dataframes with descriptive filenames
for i, (df, year_quarter) in enumerate(zip(datas, years_quarter)):
    if df is not None:
        filename = f"Financial_Report_{year_quarter}.csv"
        # Save the DataFrame
        save_dataframe(df, os.path.join(data_dir, filename))

Successfully saved dataframe to: downloaded_dataframes\Financial_Report_2023_Q4.csv
Successfully saved dataframe to: downloaded_dataframes\Financial_Report_2023_Q3.csv
Successfully saved dataframe to: downloaded_dataframes\Financial_Report_2023_Q2.csv
Successfully saved dataframe to: downloaded_dataframes\Financial_Report_2023_Q1.csv
Successfully saved dataframe to: downloaded_dataframes\Financial_Report_2022_Q4.csv
Successfully saved dataframe to: downloaded_dataframes\Financial_Report_2022_Q3.csv
Successfully saved dataframe to: downloaded_dataframes\Financial_Report_2022_Q2.csv
Successfully saved dataframe to: downloaded_dataframes\Financial_Report_2022_Q1.csv
Successfully saved dataframe to: downloaded_dataframes\Financial_Report_2021_Q4.csv
Successfully saved dataframe to: downloaded_dataframes\Financial_Report_2021_Q3.csv
Successfully saved dataframe to: downloaded_dataframes\Financial_Report_2021_Q2.csv
Successfully saved dataframe to: downloaded_dataframes\Financial_Report_2021

In [7]:
# Define a base path for downloaded dataframes (modify as needed)
data_dir = "downloaded_dataframes"

# List of filenames (modify based on your actual file naming convention)
filenames = [f"Financial_Report_{i}.csv"  for i in years_quarter]

# Dictionary to store DataFrames with names (modify naming convention)
dataframes_dict = {}  # Key: dataframe name, Value: DataFrame object

for filename in filenames:
    # Extract a base name from the filename (modify the logic if needed)
    dataframe_name = os.path.splitext(filename)[0]  # Get filename without extension

    # Construct the full path to the CSV file
    file_path = os.path.join(data_dir, filename)

    # Check if the file exists before reading
    if os.path.exists(file_path):
        try:
            df = pd.read_csv(file_path, index_col= False)  # Read CSV, set first column as index
            df = df.T.reset_index()
             # **Handle duplicates and set headers within the loop:**
            df.drop_duplicates(inplace=True)  # Remove duplicate rows
            df.columns = df.iloc[0]  # Set first row as headers
            df = df.iloc[1:]  # Remove the former first row (now headers)
            dataframes_dict[dataframe_name] = df  # Store DataFrame with name
            print(f"Successfully read: {filename} (named {dataframe_name})")
        except FileNotFoundError:
            print(f"Error: File not found: {filename}")
        except pd.errors.ParserError as e:
            print(f"Error parsing CSV: {filename}. Error details: {e}")
    else:
        print(f"Warning: Skipping {filename} as it doesn't exist in the data directory.")



Successfully read: Financial_Report_2023_Q4.csv (named Financial_Report_2023_Q4)
Successfully read: Financial_Report_2023_Q3.csv (named Financial_Report_2023_Q3)
Successfully read: Financial_Report_2023_Q2.csv (named Financial_Report_2023_Q2)
Successfully read: Financial_Report_2023_Q1.csv (named Financial_Report_2023_Q1)
Successfully read: Financial_Report_2022_Q4.csv (named Financial_Report_2022_Q4)
Successfully read: Financial_Report_2022_Q3.csv (named Financial_Report_2022_Q3)
Successfully read: Financial_Report_2022_Q2.csv (named Financial_Report_2022_Q2)
Successfully read: Financial_Report_2022_Q1.csv (named Financial_Report_2022_Q1)
Successfully read: Financial_Report_2021_Q4.csv (named Financial_Report_2021_Q4)
Successfully read: Financial_Report_2021_Q3.csv (named Financial_Report_2021_Q3)
Successfully read: Financial_Report_2021_Q2.csv (named Financial_Report_2021_Q2)
Successfully read: Financial_Report_2021_Q1.csv (named Financial_Report_2021_Q1)
Successfully read: Financial

In [8]:


for key, df in dataframes_dict.items():
    df = df.reset_index(drop = True)  # Reset index, dropping the old one
    dataframes_dict[key] = df


dataframes_dict['Financial_Report_2022_Q1']

,"CONSOLIDATED STATEMENTS OF INCOME - USD ($) shares in Millions, $ in Millions",NaN,Net Operating Revenues,Cost of goods sold,Gross Profit,"Selling, general and administrative expenses",Other operating charges,Operating Income,Interest income,Interest expense,...,Income Before Income Taxes,Income taxes,Consolidated Net Income,Less: Net income (loss) attributable to noncontrolling interests,Net Income Attributable to Shareowners of The Coca-Cola Company,Basic Net Income Per Share1,Diluted Net Income Per Share1,Average Shares Outstanding — Basic,Effect of dilutive securities,Average Shares Outstanding — Diluted
0,12 Months Ended,"Dec. 31, 2021",38655,15357,23298,12144,846,10308,276,1597,...,12425,2621,9804,33,9771,2.26,2.25,4315,25,4340
1,Unnamed: 2,"Dec. 31, 2020",33014,13433,19581,9731,853,8997,370,1437,...,9749,1981,7768,21,7747,1.8,1.79,4295,28,4323
2,Unnamed: 3,"Dec. 31, 2019",37266,14619,22647,12103,458,10086,563,946,...,10786,1801,8985,65,8920,2.09,2.07,4276,38,4314
